In [ ]:
# default_exp deepzoom

# Converting to deep zoom package 

Converting an image into a deep zoom pyramid and publishing as deep zoom package in a b2 bucket, is a three step process: 1) convert to dzi pyramid, 2) upload pyramid to bucket, 3) create html and jsonp viewers.   

In [ ]:
#hide
cd /home/frank/Work/Werkmappen_Software/dzi2b2 

/mnt/datadisk/Work/Werkmappen_Software/dzi2b2


In [ ]:
#hide
fname1 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-22_highres.png' 
fname2 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-51_highres.png'

In [ ]:
#hide
import myb2keys

application_key = myb2keys.application_key 
application_key_id = myb2keys.application_key_id 
bucket_name = myb2keys.bucket_name 

In [ ]:
#export 
import pyvips 
import os 
import shutil 
import glob
import re 
from bs4 import BeautifulSoup 
import sys
import time 
import b2sdk.v1 as b2

ROOTDIR = 'deepzoom'

class DeepZoomStore: 
    
    def __init__(self): 
        '''Create local deep zoom store directory for one or more images. '''
        
        store_dir = os.path.join('.', ROOTDIR)
        os.makedirs(store_dir, exist_ok=True) 
        
        self.images = []

        
    def dzp_save(self, fname): 
        
        name = dzp_save(fname) 
        self.images.append(name)
        
        
    def connect_b2_bucket(self, application_key_id, application_key, bucket_name): 
        
        self.bucket_name = bucket_name 
        self.b2_api, self.base_url = connect_b2_bucket(application_key_id, application_key, bucket_name)
        
        
    def make_webviewers(self): 
        
        for name in self.images: 
        
            dzi_to_js(name, self.base_url, self.bucket_name)
            make_html(name, self.bucket_name) # single image viewer 


    def upload(self):
        
        upload(self.b2_api, self.bucket_name)
        
                   

def upload(b2_api, bucket_name):
    ''''''

    source = os.path.join('.', ROOTDIR)
    destination = f'b2://{bucket_name}/{ROOTDIR}'  

    source = b2.parse_sync_folder(source, b2_api)
    destination = b2.parse_sync_folder(destination, b2_api)

    policies_manager = b2.ScanPoliciesManager(exclude_all_symlinks=True)

    synchronizer = b2.Synchronizer(
            max_workers=10,
            policies_manager=policies_manager,
            dry_run=False,
            allow_empty_source=True,
        )

    no_progress = False 

    # need to run this to actually synchronize 

    with b2.SyncReport(sys.stdout, no_progress) as reporter:
            synchronizer.sync_folders(
                source_folder=source,
                dest_folder=destination,
                now_millis=int(round(time.time() * 1000)),
                reporter=reporter,
            ) 
            
        
    

def make_html(name, bucket_name): 
    
    html_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}_view.html') 
    
    html = f'''
<!DOCTYPE html>
<html>
<body style="width: 800px; height: 600px; background-color: snow">

<div id="openseadragon1" style="width: 100%; height: 100%; background-color: gray"></div> 

<script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/openseadragon.min.js" 
        integrity="sha512-qvQYH6mPuE46uFcWLI8BdGaJpB5taX4lltbSIw5GF4iODh2xIgyz5ii1WpuzPFUknHCps0mi4mFGR44bjdZlZg==" 
        crossorigin="anonymous">
</script>

<script type="text/javascript">
var viewer = OpenSeadragon({{
    id: "openseadragon1",
    prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
    tileSources: "{name}.dzi", 
 }});
</script>


</body>
</html>  
'''
    with open(html_fname, 'w') as fh: 
        
        print(f'Saving: {html_fname}')
        fh.writelines(html)
       
    
    
def dzi_to_js(name, base_url, bucket_name): 
    '''Parse XML.dzi file into JSONP.js file. 
    
    '''
    
    url = f'{base_url}{bucket_name}/{ROOTDIR}/dzp_{name}/{name}_files/'  
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    jsonp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.js') 
    
    with open(dzi_fname, 'r') as fh: 
        lines = fh.read()
         
    soup = BeautifulSoup(lines, 'xml')

    Format = soup.Image.attrs['Format']
    Overlap = soup.Image.attrs['Overlap']
    TileSize = soup.Image.attrs['TileSize']
    Height = soup.Image.Size.attrs['Height'] 
    Width = soup.Image.Size.attrs['Width']

    jsonp = f'''OpenSeadragon( \
    {{
    id: "{name}", 
    prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
    tileSources: {{
        Image: {{
           xmlns:    "http://schemas.microsoft.com/deepzoom/2008",
           Url:      "{url}", 
           Format: "{Format}", 
           Overlap: "{Overlap}",  
           TileSize: "{TileSize}", 
           Size: {{
               Height: "{Height}", 
               Width: "{Width}" 
            }}
        }}
    }}
    }}); 
    '''
    
    with open(jsonp_fname, 'w') as fh: 
        print(f'Saving: {jsonp_fname}')
        fh.writelines(jsonp)
 

  

def connect_b2_bucket(application_key_id, application_key, bucket_name): 
    
    '''Creates upload connection to your Backblaze b2 bucket.'''
    
    info = b2.InMemoryAccountInfo()
    b2_api = b2.B2Api(info) 
    b2_api.authorize_account("production", application_key_id, application_key)  
    
    
    bucket = b2_api.get_bucket_by_name(bucket_name) 

    filenames = [file_info.file_name for file_info, _ in bucket.ls(show_versions=False, recursive=False)]
    urls = [b2_api.get_download_url_for_file_name(bucket_name, f) for f in filenames] 
    base_url = re.split(bucket_name, urls[0])[0] 
    
    return b2_api, base_url

# damn accidentally removed the dzp_save function  
# will abandon the python based file tree report function  

def dzp_save(fname): 
    # name = dzp_save(fname, report_tree=report_tree) 
    
    name, ext = os.path.splitext(os.path.basename(fname))
    dzp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}')
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    dzi_base = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}')
    
    os.makedirs(dzp_fname, exist_ok=True) 
    
    if not os.path.exists(dzi_fname): 
        print('hi')
        v_img = pyvips.Image.new_from_file(fname) 
        v_img.dzsave(dzi_base)
    else: 
        print(f'Not overwriting existing files: {dzi_base}')
    
    return name 

In [ ]:
#hide 
dz = DeepZoomStore()
dz.dzp_save(fname1)

dz.connect_b2_bucket(application_key_id, application_key, bucket_name)
dz.make_webviewers()
dz.upload()

Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js                  
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html           


Works! 